# Сценарии аварийных ситуаций генгерация данных

In [ ]:
# -*- coding: utf-8 -*-
"""
Generate an hourly table of KPI deviations for a month.
Each column is named as: [Раздел] [Показатель]
Values are percentages (float), same for every hour of the month (month-level deltas expanded to hours).

Output:
  1) hourly_deviation_df  — numeric values (percents), index is hourly timestamps
  2) CSV with semicolon delimiter and two decimals: kpi_deviation_hourly_YYYY_MM.csv
  3) (optional pretty) CSV with percent sign: kpi_deviation_hourly_YYYY_MM_percent.csv
"""

from datetime import datetime
import pandas as pd

# ────────────────────────────────────────────────────────────────────────────
# 1) Define KPI list (section, indicator, percent change)
# ────────────────────────────────────────────────────────────────────────────
kpi_metrics = [
    # Производство воды
    {"section": "Производство воды", "indicator": "Объём добычи воды", "change_percent": -5.00},
    {"section": "Производство воды", "indicator": "Объём добычи воды на собственные нужды", "change_percent": -5.00},
    {"section": "Производство воды", "indicator": "Объём подачи", "change_percent": -5.56},
    {"section": "Производство воды", "indicator": "Объём реализации", "change_percent": -1.43},
    {"section": "Производство воды", "indicator": "Объём недоучтённой воды", "change_percent": -17.50},
    {"section": "Производство воды", "indicator": "Доля недоучтённой воды", "change_percent": -13.16},
    {"section": "Производство воды", "indicator": "Общее потребление ГСМ", "change_percent": -10.00},

    # Энергопотребление
    {"section": "Энергопотребление", "indicator": "Объём электроэнергии (вода)", "change_percent": 8.33},
    {"section": "Энергопотребление", "indicator": "Удельный объём электроэнергии (вода)", "change_percent": 14.04},
    {"section": "Энергопотребление", "indicator": "Общий объём электроэнергии", "change_percent": 1.82},

    # Водоподготовка
    {"section": "Водоподготовка", "indicator": "Расход хлора", "change_percent": 0.00},
    {"section": "Водоподготовка", "indicator": "Расход коагулянта", "change_percent": 0.00},

    # Аварийность и качество (вода)
    {"section": "Аварийность и качество (вода)", "indicator": "Количество аварий (вода)", "change_percent": -20.00},
    {"section": "Аварийность и качество (вода)", "indicator": "Количество жалоб (вода)", "change_percent": -20.00},

    # Стоки
    {"section": "Стоки", "indicator": "Объём стоков", "change_percent": 20.00},
    {"section": "Стоки", "indicator": "Объём электроэнергии (стоки)", "change_percent": -6.00},
    {"section": "Стоки", "indicator": "Удельный объём электроэнергии (стоки)", "change_percent": -21.66},
    {"section": "Стоки", "indicator": "Количество засоров", "change_percent": -20.00},
    {"section": "Стоки", "indicator": "Количество аварий", "change_percent": 20.00},

    # Финансы
    {"section": "Финансы", "indicator": "Дебиторская задолженность", "change_percent": 0.00},
    {"section": "Финансы", "indicator": "Состояние расчётных счетов", "change_percent": -16.67},
    {"section": "Финансы", "indicator": "Себестоимость", "change_percent": -2.00},
]

# ────────────────────────────────────────────────────────────────────────────
# 2) Helper to build column names like: [Раздел] [Показатель]
# ────────────────────────────────────────────────────────────────────────────
def build_column_name(section: str, indicator: str) -> str:
    return f"[{section}] [{indicator}]"

# ────────────────────────────────────────────────────────────────────────────
# 3) Build hourly index for the target month
#    By default, use the last fully completed month relative to today.
# ────────────────────────────────────────────────────────────────────────────
def get_last_full_month_year_and_month(today_dt: datetime) -> tuple[int, int]:
    if today_dt.month == 1:
        return today_dt.year - 1, 12
    return today_dt.year, today_dt.month - 1

def build_hourly_index_for_month(year: int, month: int) -> pd.DatetimeIndex:
    # start: first day of month 00:00; end: first day of next month 00:00 (left-inclusive)
    next_month = 1 if month == 12 else month + 1
    next_month_year = year + 1 if month == 12 else year
    start_ts = datetime(year, month, 1, 0, 0, 0)
    end_ts = datetime(next_month_year, next_month, 1, 0, 0, 0)
    return pd.date_range(start=start_ts, end=end_ts, freq="H", inclusive="left")

# ────────────────────────────────────────────────────────────────────────────
# 4) Generate the hourly deviation table
# ────────────────────────────────────────────────────────────────────────────
def generate_hourly_deviation_table(target_year: int, target_month: int) -> pd.DataFrame:
    hourly_index: pd.DatetimeIndex = build_hourly_index_for_month(target_year, target_month)

    # Prepare columns in the order they are listed in kpi_metrics
    column_names_in_order = [build_column_name(x["section"], x["indicator"]) for x in kpi_metrics]

    # Create a dict of columns -> constant Series (expanded to all hours)
    data_columns = {}
    for metric in kpi_metrics:
        col_name = build_column_name(metric["section"], metric["indicator"])
        percent_value = float(metric["change_percent"])
        data_columns[col_name] = pd.Series([percent_value] * len(hourly_index), index=hourly_index)

    hourly_deviation_df = pd.DataFrame(data_columns, index=hourly_index)
    # Ensure column order
    hourly_deviation_df = hourly_deviation_df[column_names_in_order]
    hourly_deviation_df.index.name = "timestamp_hour"

    return hourly_deviation_df

# ────────────────────────────────────────────────────────────────────────────
# 5) Run generation for the last full month and save to CSV
# ────────────────────────────────────────────────────────────────────────────
today_now = datetime.now()
target_year, target_month = get_last_full_month_year_and_month(today_now)

hourly_deviation_df: pd.DataFrame = generate_hourly_deviation_table(target_year, target_month)

# Save numeric CSV with semicolon delimiter and two decimals (no percent sign)
csv_filename_numeric = f"kpi_deviation_hourly_{target_year}_{str(target_month).zfill(2)}.csv"
hourly_deviation_df.to_csv(csv_filename_numeric, sep=";", float_format="%.2f", encoding="utf-8")

# Save a "pretty" CSV with percent signs (strings) — optional
pretty_df = hourly_deviation_df.copy().applymap(lambda v: f"{v:.2f}%")
csv_filename_pretty = f"kpi_deviation_hourly_{target_year}_{str(target_month).zfill(2)}_percent.csv"
pretty_df.to_csv(csv_filename_pretty, sep=";", encoding="utf-8")

# Show a quick preview in notebooks (safe to ignore if running as a script)
try:
    from IPython.display import display  # type: ignore
    print(f"Hourly rows: {hourly_deviation_df.shape[0]}, KPI columns: {hourly_deviation_df.shape[1]}")
    print("Preview (first 8 rows, numeric):")
    display(hourly_deviation_df.head(8).round(2))
    print("Saved files:")
    print(" -", csv_filename_numeric)
    print(" -", csv_filename_pretty)
except Exception:
    # Non-notebook environments will simply skip display
    pass


Hourly rows: 720, KPI columns: 22
Preview (first 8 rows, numeric):


/tmp/ipython-input-2563210170.py:76: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.date_range(start=start_ts, end=end_ts, freq="H", inclusive="left")
/tmp/ipython-input-2563210170.py:114: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pretty_df = hourly_deviation_df.copy().applymap(lambda v: f"{v:.2f}%")


,[Производство воды] [Объём добычи воды],[Производство воды] [Объём добычи воды на собственные нужды],[Производство воды] [Объём подачи],[Производство воды] [Объём реализации],[Производство воды] [Объём недоучтённой воды],[Производство воды] [Доля недоучтённой воды],[Производство воды] [Общее потребление ГСМ],[Энергопотребление] [Объём электроэнергии (вода)],[Энергопотребление] [Удельный объём электроэнергии (вода)],[Энергопотребление] [Общий объём электроэнергии],...,[Аварийность и качество (вода)] [Количество аварий (вода)],[Аварийность и качество (вода)] [Количество жалоб (вода)],[Стоки] [Объём стоков],[Стоки] [Объём электроэнергии (стоки)],[Стоки] [Удельный объём электроэнергии (стоки)],[Стоки] [Количество засоров],[Стоки] [Количество аварий],[Финансы] [Дебиторская задолженность],[Финансы] [Состояние расчётных счетов],[Финансы] [Себестоимость]
timestamp_hour,,,,,,,,,,,,,,,,,,,,,
2025-09-01 00:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 01:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 02:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 03:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 04:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 05:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 06:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 07:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0


Saved files:
 - kpi_deviation_hourly_2025_09.csv
 - kpi_deviation_hourly_2025_09_percent.csv


In [ ]:
# -*- coding: utf-8 -*-
"""
Simulate hourly KPI deviations for each emergency scenario over the last full month.

What you get:
  • Baseline hourly table with columns named as: "[Раздел] [Показатель]"
  • For each scenario (14 total) — a separate hourly CSV with the simulated situation
  • A manifest CSV describing when each scenario starts/ends, duration and severity

CSV format: semicolon-delimited, two decimals, UTF-8.

No placeholders. All variable names are in English. Column captions are Russian as requested.
"""

from __future__ import annotations

from datetime import datetime
from typing import Dict, List, Tuple
import os
import math
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────────────────
# 1) Baseline: month-level KPI deltas expanded to hourly timestamps
# ────────────────────────────────────────────────────────────────────────────

kpi_metrics: List[Dict[str, object]] = [
    # Производство воды
    {"section": "Производство воды", "indicator": "Объём добычи воды", "change_percent": -5.00},
    {"section": "Производство воды", "indicator": "Объём добычи воды на собственные нужды", "change_percent": -5.00},
    {"section": "Производство воды", "indicator": "Объём подачи", "change_percent": -5.56},
    {"section": "Производство воды", "indicator": "Объём реализации", "change_percent": -1.43},
    {"section": "Производство воды", "indicator": "Объём недоучтённой воды", "change_percent": -17.50},
    {"section": "Производство воды", "indicator": "Доля недоучтённой воды", "change_percent": -13.16},
    {"section": "Производство воды", "indicator": "Общее потребление ГСМ", "change_percent": -10.00},

    # Энергопотребление
    {"section": "Энергопотребление", "indicator": "Объём электроэнергии (вода)", "change_percent": 8.33},
    {"section": "Энергопотребление", "indicator": "Удельный объём электроэнергии (вода)", "change_percent": 14.04},
    {"section": "Энергопотребление", "indicator": "Общий объём электроэнергии", "change_percent": 1.82},

    # Водоподготовка
    {"section": "Водоподготовка", "indicator": "Расход хлора", "change_percent": 0.00},
    {"section": "Водоподготовка", "indicator": "Расход коагулянта", "change_percent": 0.00},

    # Аварийность и качество (вода)
    {"section": "Аварийность и качество (вода)", "indicator": "Количество аварий (вода)", "change_percent": -20.00},
    {"section": "Аварийность и качество (вода)", "indicator": "Количество жалоб (вода)", "change_percent": -20.00},

    # Стоки
    {"section": "Стоки", "indicator": "Объём стоков", "change_percent": 20.00},
    {"section": "Стоки", "indicator": "Объём электроэнергии (стоки)", "change_percent": -6.00},
    {"section": "Стоки", "indicator": "Удельный объём электроэнергии (стоки)", "change_percent": -21.66},
    {"section": "Стоки", "indicator": "Количество засоров", "change_percent": -20.00},
    {"section": "Стоки", "indicator": "Количество аварий", "change_percent": 20.00},

    # Финансы
    {"section": "Финансы", "indicator": "Дебиторская задолженность", "change_percent": 0.00},
    {"section": "Финансы", "indicator": "Состояние расчётных счетов", "change_percent": -16.67},
    {"section": "Финансы", "indicator": "Себестоимость", "change_percent": -2.00},
]

def build_column_name(section: str, indicator: str) -> str:
    return f"[{section}] [{indicator}]"

def get_last_full_year_month(now_dt: datetime) -> Tuple[int, int]:
    if now_dt.month == 1:
        return now_dt.year - 1, 12
    return now_dt.year, now_dt.month - 1

def build_hourly_index_for_month(year: int, month: int) -> pd.DatetimeIndex:
    next_month = 1 if month == 12 else month + 1
    next_year = year + 1 if month == 12 else year
    start_ts = datetime(year, month, 1, 0, 0, 0)
    end_ts = datetime(next_year, next_month, 1, 0, 0, 0)
    return pd.date_range(start=start_ts, end=end_ts, freq="H", inclusive="left")

def generate_baseline_hourly_table(target_year: int, target_month: int) -> pd.DataFrame:
    hourly_index = build_hourly_index_for_month(target_year, target_month)
    column_order = [build_column_name(x["section"], x["indicator"]) for x in kpi_metrics]
    data: Dict[str, pd.Series] = {}
    for metric in kpi_metrics:
        col = build_column_name(metric["section"], metric["indicator"])
        value = float(metric["change_percent"])
        data[col] = pd.Series([value] * len(hourly_index), index=hourly_index)
    baseline_df = pd.DataFrame(data, index=hourly_index)
    baseline_df = baseline_df[column_order]
    baseline_df.index.name = "timestamp_hour"
    return baseline_df

# ────────────────────────────────────────────────────────────────────────────
# 2) Scenario engine
#    • Each scenario: start index (deterministic), duration, severity
#    • Impacts: additive deltas in percentage points on selected columns
#    • Shape: "step" or "triangle"
# ────────────────────────────────────────────────────────────────────────────

def slugify(text: str) -> str:
    # simple ASCII slug (keeps digits/latin/underscore, replaces spaces)
    mapping = {
        " ": "_", "(": "", ")": "", "—": "-", "–": "-", "/": "-", ",": "", ".": "",
        "№": "N", "%": "pct", ":": "", ";": "", "«": "", "»": "", "[": "", "]": ""
    }
    for k, v in mapping.items():
        text = text.replace(k, v)
    return "".join(ch for ch in text if ch.isalnum() or ch in "_-").lower()

def pick_start_index(hourly_index: pd.DatetimeIndex, scenario_index: int, duration_hours: int) -> int:
    # Spread starts every 48 hours; keep inside bounds
    total_hours = len(hourly_index)
    max_start = max(0, total_hours - duration_hours - 1)
    candidate = (scenario_index * 48) % (max_start + 1 if max_start > 0 else 1)
    return int(candidate)

def shape_vector(length: int, shape: str) -> np.ndarray:
    if length <= 0:
        return np.array([], dtype=float)
    if shape == "step":
        return np.ones(length, dtype=float)
    if shape == "triangle":
        # Triangle 0 → 1 → 0 across the window
        x = np.linspace(-1.0, 1.0, length)
        return 1.0 - np.abs(x)
    # Fallback: step
    return np.ones(length, dtype=float)

def apply_impacts(
    frame: pd.DataFrame,
    start_idx: int,
    duration_hours: int,
    impacts: Dict[str, float],
    shape: str = "step"
) -> pd.DataFrame:
    result = frame.copy()
    end_idx = start_idx + duration_hours
    window = slice(frame.index[start_idx], frame.index[min(end_idx, len(frame.index)-1)])
    # Build vector
    vec = shape_vector(duration_hours, shape)
    # Align length (slice above is inclusive, adjust to exact duration)
    window_index = frame.index[start_idx:start_idx + duration_hours]
    if len(window_index) != duration_hours:
        vec = shape_vector(len(window_index), shape)

    for col, delta in impacts.items():
        if col not in result.columns:
            raise KeyError(f"Column not found: {col}")
        series = result.loc[window_index, col].to_numpy(dtype=float)
        series = series + vec * float(delta)
        result.loc[window_index, col] = series
    return result

# ────────────────────────────────────────────────────────────────────────────
# 3) Scenario catalog — impacts are additive percentage-point deltas
# ────────────────────────────────────────────────────────────────────────────

C = {
    "supply": "[Производство воды] [Объём подачи]",
    "sales": "[Производство воды] [Объём реализации]",
    "nrw_vol": "[Производство воды] [Объём недоучтённой воды]",
    "nrw_share": "[Производство воды] [Доля недоучтённой воды]",
    "fuel_total": "[Производство воды] [Общее потребление ГСМ]",
    "e_water": "[Энергопотребление] [Объём электроэнергии (вода)]",
    "e_water_sp": "[Энергопотребление] [Удельный объём электроэнергии (вода)]",
    "e_total": "[Энергопотребление] [Общий объём электроэнергии]",
    "chlorine": "[Водоподготовка] [Расход хлора]",
    "coagulant": "[Водоподготовка] [Расход коагулянта]",
    "incidents_w": "[Аварийность и качество (вода)] [Количество аварий (вода)]",
    "complaints_w": "[Аварийность и качество (вода)] [Количество жалоб (вода)]",
    "sew_flow": "[Стоки] [Объём стоков]",
    "e_sew": "[Стоки] [Объём электроэнергии (стоки)]",
    "e_sew_sp": "[Стоки] [Удельный объём электроэнергии (стоки)]",
    "clogs": "[Стоки] [Количество засоров]",
    "incidents_sew": "[Стоки] [Количество аварий]",
    "debts": "[Финансы] [Дебиторская задолженность]",
    "accounts": "[Финансы] [Состояние расчётных счетов]",
    "cost": "[Финансы] [Себестоимость]",
}

scenarios_spec: List[Dict[str, object]] = [
    {
        "name": "Прорыв магистрального водовода",
        "duration_hours": 12,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["supply"]: -30.0,
            C["sales"]: -40.0,
            C["nrw_vol"]: +80.0,
            C["nrw_share"]: +80.0,
            C["incidents_w"]: +100.0,
            C["complaints_w"]: +150.0,
        },
    },
    {
        "name": "Отказ насосной станции",
        "duration_hours": 8,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["supply"]: -20.0,
            C["e_water_sp"]: +15.0,
            C["incidents_w"]: +50.0,
        },
    },
    {
        "name": "Отключение электропитания, ДГУ запускается поздно",
        "duration_hours": 10,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["supply"]: -70.0,
            C["fuel_total"]: +150.0,
            C["incidents_w"]: +50.0,
        },
    },
    {
        "name": "Срыв коагуляции (мутность)",
        "duration_hours": 18,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["coagulant"]: +50.0,
            C["e_water"]: +10.0,
            C["complaints_w"]: +60.0,
        },
    },
    {
        "name": "Срыв дезинфекции (остаточный хлор вне нормы)",
        "duration_hours": 6,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["chlorine"]: +80.0,
            C["complaints_w"]: +80.0,
            C["incidents_w"]: +40.0,
        },
    },
    {
        "name": "Загрязнение источника (нефтепродукты/токсины)",
        "duration_hours": 24,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["coagulant"]: +70.0,
            C["chlorine"]: +50.0,
            C["complaints_w"]: +60.0,
            C["sales"]: -10.0,
        },
    },
    {
        "name": "Ливневый приток в канализацию (риск переливов)",
        "duration_hours": 20,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["sew_flow"]: +120.0,
            C["e_sew"]: +60.0,
            C["incidents_sew"]: +80.0,
            C["clogs"]: +50.0,
        },
    },
    {
        "name": "Засор коллектора",
        "duration_hours": 14,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["clogs"]: +200.0,
            C["sew_flow"]: -30.0,
            C["e_sew"]: -20.0,
            C["incidents_sew"]: +100.0,
        },
    },
    {
        "name": "Утечка хлора (газ) на хлораторной",
        "duration_hours": 5,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["chlorine"]: -80.0,
            C["incidents_w"]: +100.0,
            C["complaints_w"]: +70.0,
        },
    },
    {
        "name": "Рост недоучтённой воды (NRW) в секторе",
        "duration_hours": 36,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["nrw_vol"]: +150.0,
            C["nrw_share"]: +150.0,
            C["sales"]: -10.0,
        },
    },
    {
        "name": "Киберинцидент/сбой SCADA",
        "duration_hours": 9,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["e_water_sp"]: +10.0,
            C["supply"]: -10.0,
            C["incidents_w"]: +20.0,
            C["complaints_w"]: +20.0,
        },
    },
    {
        "name": "Некачественное топливо/неисправность ДГУ (тесты)",
        "duration_hours": 7,
        "severity": 1.0,
        "shape": "step",
        "impacts": {
            C["fuel_total"]: +200.0,
            C["incidents_w"]: +20.0,
        },
    },
    {
        "name": "Сбой промывки фильтров",
        "duration_hours": 11,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["e_water_sp"]: +20.0,
            C["complaints_w"]: +50.0,
        },
    },
    {
        "name": "Контаминация резервуара (РЧВ)",
        "duration_hours": 16,
        "severity": 1.0,
        "shape": "triangle",
        "impacts": {
            C["chlorine"]: +60.0,
            C["complaints_w"]: +70.0,
            C["sales"]: -5.0,
        },
    },
]

# ────────────────────────────────────────────────────────────────────────────
# 4) Run generation
# ────────────────────────────────────────────────────────────────────────────

now_dt = datetime.now()
target_year, target_month = get_last_full_year_month(now_dt)
baseline_df: pd.DataFrame = generate_baseline_hourly_table(target_year, target_month)

output_dir = os.path.abspath(".")
baseline_csv = f"kpi_deviation_hourly_{target_year}_{str(target_month).zfill(2)}.csv"
baseline_df.to_csv(baseline_csv, sep=";", float_format="%.2f", encoding="utf-8")

manifest_records: List[Dict[str, object]] = []

for idx, spec in enumerate(scenarios_spec):
    scenario_name: str = str(spec["name"])
    duration_hours: int = int(spec["duration_hours"])
    severity: float = float(spec["severity"])
    shape: str = str(spec["shape"])
    impacts_raw: Dict[str, float] = dict(spec["impacts"])

    # scale impacts by severity (if severity != 1.0)
    impacts_scaled: Dict[str, float] = {k: v * severity for k, v in impacts_raw.items()}

    start_idx = pick_start_index(baseline_df.index, scenario_index=idx, duration_hours=duration_hours)
    scenario_df = apply_impacts(
        frame=baseline_df,
        start_idx=start_idx,
        duration_hours=duration_hours,
        impacts=impacts_scaled,
        shape=shape,
    )

    start_ts = baseline_df.index[start_idx]
    end_ts = baseline_df.index[min(start_idx + duration_hours - 1, len(baseline_df.index) - 1)]

    scenario_slug = slugify(scenario_name)
    scenario_csv = f"scenario_{str(idx+1).zfill(2)}_{scenario_slug}_hourly_{target_year}_{str(target_month).zfill(2)}.csv"
    scenario_df.to_csv(scenario_csv, sep=";", float_format="%.2f", encoding="utf-8")

    manifest_records.append({
        "scenario_index": idx + 1,
        "scenario_name": scenario_name,
        "scenario_slug": scenario_slug,
        "year": target_year,
        "month": target_month,
        "duration_hours": duration_hours,
        "shape": shape,
        "severity": severity,
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "csv_file": scenario_csv,
    })

manifest_df = pd.DataFrame(manifest_records)
manifest_csv = f"scenarios_manifest_{target_year}_{str(target_month).zfill(2)}.csv"
manifest_df.to_csv(manifest_csv, sep=";", index=False, encoding="utf-8")

# Optional quick preview when running in notebooks
try:
    from IPython.display import display  # type: ignore
    print(f"Baseline saved to: {baseline_csv}")
    print(f"Scenarios manifest saved to: {manifest_csv}")
    print("scenarios:")
    display(manifest_df)
    print("Baseline preview (first 6 rows):")
    display(baseline_df.head(6).round(2))
except Exception:
    pass


/tmp/ipython-input-2328706022.py:77: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.date_range(start=start_ts, end=end_ts, freq="H", inclusive="left")


Baseline saved to: kpi_deviation_hourly_2025_09.csv
Scenarios manifest saved to: scenarios_manifest_2025_09.csv
scenarios:


,scenario_index,scenario_name,scenario_slug,year,month,duration_hours,shape,severity,start_timestamp,end_timestamp,csv_file
0,1,Прорыв магистрального водовода,прорыв_магистрального_водовода,2025,9,12,triangle,1.0,2025-09-01,2025-09-01 11:00:00,scenario_01_прорыв_магистрального_водовода_hou...
1,2,Отказ насосной станции,отказ_насосной_станции,2025,9,8,step,1.0,2025-09-03,2025-09-03 07:00:00,scenario_02_отказ_насосной_станции_hourly_2025...
2,3,"Отключение электропитания, ДГУ запускается поздно",отключение_электропитания_дгу_запускается_поздно,2025,9,10,triangle,1.0,2025-09-05,2025-09-05 09:00:00,scenario_03_отключение_электропитания_дгу_запу...
3,4,Срыв коагуляции (мутность),срыв_коагуляции_мутность,2025,9,18,step,1.0,2025-09-07,2025-09-07 17:00:00,scenario_04_срыв_коагуляции_мутность_hourly_20...
4,5,Срыв дезинфекции (остаточный хлор вне нормы),срыв_дезинфекции_остаточный_хлор_вне_нормы,2025,9,6,step,1.0,2025-09-09,2025-09-09 05:00:00,scenario_05_срыв_дезинфекции_остаточный_хлор_в...
5,6,Загрязнение источника (нефтепродукты/токсины),загрязнение_источника_нефтепродукты-токсины,2025,9,24,triangle,1.0,2025-09-11,2025-09-11 23:00:00,scenario_06_загрязнение_источника_нефтепродукт...
6,7,Ливневый приток в канализацию (риск переливов),ливневый_приток_в_канализацию_риск_переливов,2025,9,20,triangle,1.0,2025-09-13,2025-09-13 19:00:00,scenario_07_ливневый_приток_в_канализацию_риск...
7,8,Засор коллектора,засор_коллектора,2025,9,14,step,1.0,2025-09-15,2025-09-15 13:00:00,scenario_08_засор_коллектора_hourly_2025_09.csv
8,9,Утечка хлора (газ) на хлораторной,утечка_хлора_газ_на_хлораторной,2025,9,5,triangle,1.0,2025-09-17,2025-09-17 04:00:00,scenario_09_утечка_хлора_газ_на_хлораторной_ho...
9,10,Рост недоучтённой воды (NRW) в секторе,рост_недоучтённой_воды_nrw_в_секторе,2025,9,36,step,1.0,2025-09-19,2025-09-20 11:00:00,scenario_10_рост_недоучтённой_воды_nrw_в_секто...


Baseline preview (first 6 rows):


,[Производство воды] [Объём добычи воды],[Производство воды] [Объём добычи воды на собственные нужды],[Производство воды] [Объём подачи],[Производство воды] [Объём реализации],[Производство воды] [Объём недоучтённой воды],[Производство воды] [Доля недоучтённой воды],[Производство воды] [Общее потребление ГСМ],[Энергопотребление] [Объём электроэнергии (вода)],[Энергопотребление] [Удельный объём электроэнергии (вода)],[Энергопотребление] [Общий объём электроэнергии],...,[Аварийность и качество (вода)] [Количество аварий (вода)],[Аварийность и качество (вода)] [Количество жалоб (вода)],[Стоки] [Объём стоков],[Стоки] [Объём электроэнергии (стоки)],[Стоки] [Удельный объём электроэнергии (стоки)],[Стоки] [Количество засоров],[Стоки] [Количество аварий],[Финансы] [Дебиторская задолженность],[Финансы] [Состояние расчётных счетов],[Финансы] [Себестоимость]
timestamp_hour,,,,,,,,,,,,,,,,,,,,,
2025-09-01 00:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 01:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 02:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 03:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 04:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0
2025-09-01 05:00:00,-5.0,-5.0,-5.56,-1.43,-17.5,-13.16,-10.0,8.33,14.04,1.82,...,-20.0,-20.0,20.0,-6.0,-21.66,-20.0,20.0,0.0,-16.67,-2.0


In [5]:
# Save all CSV files from the current folder into a ZIP archive (non-recursive)

from pathlib import Path
from datetime import datetime
import zipfile

# ───────────── Settings ─────────────
search_directory: Path = Path.cwd()            # current working directory
file_extension: str = ".csv"                   # target extension
use_deflate_compression: bool = True           # ZIP_DEFLATED for smaller size
compression_level: int = 9                     # 0..9 (only used for ZIP_DEFLATED)

# ───────────── Collect CSV files ─────────────
csv_files_in_directory = sorted(
    [p for p in search_directory.iterdir() if p.is_file() and p.suffix.lower() == file_extension]
)

if not csv_files_in_directory:
    print("No CSV files found in the current folder.")
else:
    # ───────────── Build archive name ─────────────
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_filename = f"vodokanal_gen_{timestamp_str}.zip"
    archive_path = search_directory / archive_filename

    # ───────────── Choose compression ─────────────
    zip_compression = zipfile.ZIP_DEFLATED if use_deflate_compression else zipfile.ZIP_STORED

    # ───────────── Create ZIP ─────────────
    with zipfile.ZipFile(
        archive_path,
        mode="w",
        compression=zip_compression,
        compresslevel=(compression_level if zip_compression == zipfile.ZIP_DEFLATED else None),
    ) as zip_file:
        for file_path in csv_files_in_directory:
            # arcname stores only the filename inside the archive (no absolute paths)
            zip_file.write(filename=file_path, arcname=file_path.name)

    print(f"Saved {len(csv_files_in_directory)} CSV files to archive: {archive_path}")


Saved 17 CSV files to archive: /content/vodokanal_gen_20251002_070048.zip
